In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [32]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.0 GB  | Proc size: 3.8 GB
GPU RAM Free: 14377MB | Used: 1903MB | Util  12% | Total 16280MB


In [33]:
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jvx8olsn
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-jvx8olsn
  Created wheel for transformers: filename=transformers-2.8.0-cp36-none-any.whl size=602051 sha256=54c22238ee1a3c9ab667c1a3962bc4f5196b117eb1d79dfb84d339183f3a76eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-vzzyrwgu/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers


In [34]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Overwriting setup.sh


In [35]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
Created temporary directory: /tmp/pip-ephem-wheel-cache-bl7c5yc4
Created temporary directory: /tmp/pip-req-tracker-xbe_wr9d
Created requirements tracker '/tmp/pip-req-tracker-xbe_wr9d'
Created temporary directory: /tmp/pip-install-10bu_v86
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-_wr72rj2
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-xbe_wr9d'
    Running setup.py (path:/tmp/pip-req-build-_wr72rj2/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-_wr72rj2/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-_wr72rj2/pip-egg-info/apex.egg-info/PKG-INFO
    writing dependency_links to /tmp/pip-req-build-_wr72rj2/pip-egg-info/apex.egg-info/dependency_links.txt
    writing top-level names to /tmp/pip-req-build-_wr

In [0]:
from __future__ import absolute_import, division, print_function

import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.utils.data.distributed import DistributedSampler

try:
    from torch.utils.tensorboard import SummaryWriter
except:
    from tensorboardX import SummaryWriter

from tqdm import tqdm, trange

from transformers import (WEIGHTS_NAME, AdamW, 
                          # WarmupLinearSchedule,
                                  BertConfig, BertForMaskedLM, BertTokenizer,
                                  GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
                                  OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                                  RobertaConfig, RobertaForMaskedLM, RobertaTokenizer,
                                  DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)


logger = logging.getLogger(__name__)

In [0]:
logger.setLevel('INFO')

In [0]:
MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
    'openai-gpt': (OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    'bert': (BertConfig, BertForMaskedLM, BertTokenizer),
    'roberta': (RobertaConfig, RobertaForMaskedLM, RobertaTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)
}

In [0]:
! cp './drive/My Drive/gpt2/kanye.zip' kanye.zip

In [56]:
! rm -rf textgenmodels
! unzip kanye.zip

Archive:  kanye.zip
   creating: textgenmodels/
  inflating: textgenmodels/tf_model.h5  
  inflating: textgenmodels/training_args.bin  
 extracting: textgenmodels/tokenizer_config.json  
  inflating: textgenmodels/config.json  
 extracting: textgenmodels/eval_results.txt  
  inflating: textgenmodels/merges.txt  
  inflating: textgenmodels/pytorch_model.bin  
  inflating: textgenmodels/special_tokens_map.json  
  inflating: textgenmodels/vocab.json  


In [0]:
def dict2obj(d):
  if isinstance(d, list):
    d = [dict2obj(x) for x in d]
  if not isinstance(d, dict):
    return d
  class C(object):
    pass
  o = C()
  for k in d:
    o.__dict__[k] = dict2obj(d[k])
  return o


parser = {}

# parser['train_data_file'] = './texts/rap_lyrics.txt'
# parser['output_dir'] = './drive/My Drive/gpt2/pretrained/en_rap'
parser['output_dir'] = './textgenmodels'

parser['eval_data_file'] = False
parser['model_type'] = 'gpt2' # bert
parser['model_name_or_path'] = 'gpt2' # 'bert-base-cased'
parser['mlm'] = False 
parser['mlm_probability'] = False

parser['config_name'] = ""
parser['tokenizer_name'] = ""
parser['cache_dir'] = ""
parser['block_size'] = -1
parser['do_train'] = True
parser['do_eval'] = False
parser['evaluate_during_training'] = False
parser['do_lower_case'] = True

parser['per_gpu_train_batch_size'] = 2
parser['per_gpu_eval_batch_size'] = 1
parser['gradient_accumulation_steps'] = 10
parser['learning_rate'] = 5e-5
parser['weight_decay'] = 0.0
parser['adam_epsilon'] = 1e-8
parser['max_grad_norm'] = 1.0
parser['num_train_epochs'] = 3.0
parser['max_steps'] = -1
parser['warmup_steps'] = 0

parser['logging_steps'] = 50
parser['save_steps'] = 50
parser['save_total_limit'] = None
parser['eval_all_checkpoints'] = False
parser['no_cuda'] = False
parser['overwrite_output_dir'] = True
parser['overwrite_cache'] = True
parser['seed'] = 42

parser['fp16'] = True
parser['fp16_opt_level'] = 'O1'
parser['local_rank'] = -1
parser['server_ip'] = ""
parser['server_port'] = ""

parser['device'] = "cuda"

args = dict2obj(parser)

In [58]:
! ls './drive/My Drive/gpt2/pretrained/en_rap'

 added_tokens.json   merges.txt		       tokenizer_config.json
'config (1).json'    pytorch_model.bin	       training_args.bin
 config.json	     special_tokens_map.json   vocab.json


In [59]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]

# Load a trained model and vocabulary that you have fine-tuned
model = model_class.from_pretrained(args.output_dir)
tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
model.to(args.device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [0]:
import torch.nn.functional as F

def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (batch size x vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, context, num_samples=1, temperature=1, top_k=0, top_p=0.0, repetition_penalty=1.0,
                    is_xlnet=False, is_xlm_mlm=False, xlm_mask_token=None, xlm_lang=None, device='cuda'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context
    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            if is_xlnet: 
                # XLNet is a direct (predict same token, not next token) and bi-directional model by default
                # => need one additional dummy token in the input (will be masked), attention mask and target mapping (see model docstring)
                input_ids = torch.cat((generated, torch.zeros((1, 1), dtype=torch.long, device=device)), dim=1)
                perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float, device=device)
                perm_mask[:, :, -1] = 1.0  # Previous tokens don't see last token
                target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float, device=device)
                target_mapping[0, 0, -1] = 1.0  # predict last token
                inputs = {'input_ids': input_ids, 'perm_mask': perm_mask, 'target_mapping': target_mapping}

            if is_xlm_mlm and xlm_mask_token:
                # XLM MLM models are direct models (predict same token, not next token)
                # => need one additional dummy token in the input (will be masked and guessed)
                input_ids = torch.cat((generated, torch.full((1, 1), xlm_mask_token, dtype=torch.long, device=device)), dim=1)
                inputs = {'input_ids': input_ids}

            if xlm_lang is not None:
                inputs["langs"] = torch.tensor([xlm_lang] * inputs["input_ids"].shape[1], device=device).view(1, -1)

            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet/CTRL (cached hidden-states)
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # repetition penalty from CTRL (https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling:
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated

In [65]:
CONTEXT_TEXT = 'I wanna fuck you hard on the sink \
After that, give you somethin\' to drink \
Step back, can\'t get spunk on the mink \
I mean damn, what would Jeromey Romey Romey Rome think? \
Hey, you remember where we first met? \
Okay, I don\'t remember where we first met.\n'

# CONTEXT_TEXT = 'Bitch I\'m back out my coma \
# Wakin\' up on your sofa \
# When I park my Range Rover \
# Slightly scratch your Corolla \
# Okay, I smashed your Corolla \
# I\'m hangin\' on a hangover \n'

START_TEXT = 'Jesus '
CONTEXT_TEXT += START_TEXT

context_tokens = tokenizer.encode(CONTEXT_TEXT, add_special_tokens=False)
sampled = sample_sequence(model, 100, context_tokens, 
                          temperature = 1.0,
                          top_p = 0.9)

out = sampled[:, len(context_tokens):].tolist()
text = ''.join([tokenizer.decode(o, clean_up_tokenization_spaces=True) for o in out])
text = text[: text.find('<|endoftext|>')].split('\n')

print('-' * 20)


for i, t in enumerate(text):
  if i == 0:
    print(START_TEXT + t)
  else:
    print(t)

100%|██████████| 100/100 [00:02<00:00, 39.60it/s]

--------------------
Jesus , right, i gotta say we went to porn, money, drugs
ok, i want money
and who cares? man, these rolls i'm rollin' here
they put together a gang of new people, everybody take a picture for one week
yep, that's why i choose to be with you guys, but you just watch me like that
you know i just want more
ok, so where you gon' find me?
never again as your dick up i
